### Q learning taxi game

Be sure to install the OpenAI Gym, by running this from the terminal: 
 - pip install gym

In [ ]:
import numpy as np
import gym
import ipywidgets as widgets
import taxi_functions
from ipywidgets import interact

Open the Taxi game from OpenAI Gym

In [ ]:
env = gym.make('Taxi-v2')

Reset the environment to a random state

In [ ]:
env.reset()

Note that there are 500 possible environment states

In [ ]:
env.observation_space.n

Here is what they look like...
 - The yellow rectangle is the taxi
 - The |'s are walls
 - The :'s are road
 - The goal is to pick up someone at blue letter, then drop off at pink letter

In [ ]:
env.render()

Note that there are 6 possible actions the taxi can take at any time step: 
 - down, up, right, left, pick up, drop off

In [ ]:
env.action_space.n

Experiment with taking actions...

In [ ]:
state, reward, done, info = env.step(1)
env.render()

### Perform tabular Q-learning

For background:

 - The action-value function Q(state, action) is defined in Equation (4) of the notes.

 - The Q-learning algorithm is covered in Section 5.2 of the notes; see Equation (46).

Initialize the action-value function Q(state, action) and choose a learning rate alpha

In [ ]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
alpha = 0.1

Play the game for a certain number of episodes, updating the Q-function after each action

In [ ]:
n_episodes = 10**5
print_freq = 10 # logarithmic
prev_freq = 0

avg_step_count = 0
for episode in range(1, n_episodes+1):
    state = env.reset()
    G = 0
    step_count = 0
    
    # Let epsilon-greediness decay with episode
    epsilon = 10 / episode
    
    # Continue until taxi stumbles upon correct pick-up and drop-off (thus earning 20 points)
    reward = 0
    while reward != 20:
        
        # Choose action epsilon-greedily
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])
        
        # Perform action
        state2, reward, done, info = env.step(action)
        step_count += 1
        G += reward
        
        # Update action-value function according to Q-learning algorithm
        Q[state, action] += alpha * (reward + np.max(Q[state2]) - Q[state, action])
        
        # Update state and proceed to next action
        state = state2
        
    # Track stats
    if episode % print_freq == 0:
        avg_step_count += 1/print_freq * (step_count - avg_step_count)
        print('Episode: {}, Average Step Count: {:.2f}'.format(episode, avg_step_count))
        avg_step_count = 0
        prev_freq = print_freq
        print_freq *= 10
    else:
        avg_step_count += 1/(episode - prev_freq) * (step_count - avg_step_count)

### Results

Functions used for interactive results

In [ ]:
def run_episode(Q, epsilon):
    states = []
    actions = []
    state = env.reset()
    states.append(state)
    done = False
    while not done:
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])
        actions.append(action)
        state, reward, done, info = env.step(action)
        states.append(state)
    return states, actions

def snapshot(t):
    if t == 0:
        env.reset()
        env.env.s = states[0]
    else:
        env.env.s = states[t-1]
        env.step(actions[t-1])
    env.render()

Here is an example of the initial random strategy in action

In [ ]:
states, actions = run_episode(Q, 1)
interact(snapshot, t=widgets.IntSlider(min=0, max=len(states)-1, step=1, value=0));

Here is the learned optimal policy in action

In [ ]:
states, actions = run_episode(Q, 0)
interact(snapshot, t=widgets.IntSlider(min=0, max=len(states)-1, step=1, value=0));